# Properties and Data Descriptors

The `property` objects __are__ data descriptors. They have `__get__`, `__set__` and `__delete__` methods

In [1]:
from numbers import Integral

In [2]:
class Person:

    @property
    def age(self):
        return getattr(self, '_age', None)
    
    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise ValueError(f'Age must be an Integer')
        
        if value < 0:
            raise ValueError(f'Age must be a positive integer')
        
        self._age = value

In [3]:
p = Person()

try:
    p.age = -10
except ValueError as e:
    print(e)

Age must be a positive integer


In [4]:
p.age = 10 
p.__dict__

{'_age': 10}

Creating the same class without decorators

In [5]:
class Person:

    def get_age(self):
        return getattr(self, '_age', None)
    
    def set_age(self, value):
        if not isinstance(value, Integral):
            raise ValueError(f'Age must be an Integer')
        
        if value < 0:
            raise ValueError(f'Age must be a positive integer')
        
        self._age = value

    age = property(fget=get_age, fset=set_age)

Exatly the same thing as the class above, but in a longer way. Uses the property object class

In [7]:
property_object = Person.age

In [8]:
hasattr(property_object, '__get__')

True

In [9]:
hasattr(property_object, '__set__')

True

In [10]:
hasattr(property_object, '__delete__')

True

In properties, the dunder methods are implemented automaticaly. They're always going to have it bc they inherit it from the property object

In [11]:
p = Person()
p.age = 10 
print(p.age)

10


In [12]:
class TimeUTC:
    @property
    def current_time(self):
        return 'current_time'

In [13]:
t = TimeUTC()
hasattr(TimeUTC.current_time, '__get__')

True

In [14]:
hasattr(TimeUTC.current_time, '__set__')

True

In [15]:
t.current_time

'current_time'

In [16]:
t.current_time = '205'

AttributeError: can't set attribute 'current_time'

We get attribute error bc we didn't defined a `fset` method. The property has the `__set__` method, but the `fset` wich to call is not defined

In [17]:
p = Person()
p.__dict__

{}

In [18]:
p.age = 10

In [19]:
p.age

10

In [20]:
p.__dict__

{'_age': 10}

In [21]:
p.__dict__['age'] = 100

In [22]:
p.__dict__

{'_age': 10, 'age': 100}

In [23]:
p.age

10

In [24]:
p.__dict__['age']

100

Since `age` is a data descriptor, python is going to use the property value, not the dictionary

### Creating our own class that basically create properties

In [25]:
class MakeProperty:
    def __init__(self, fget=None, fset=None):
        self.fset = fset
        self.fget = fget 

    def __set_name__(self, instance, property_name):
        self.prop_name = property_name

    def __get__(self, instance, owner_class):
        print('__get__ called')
        if instance is None:
            return self
        
        if self.fget is None:
            raise AttributeError(f'{self.prop_name} is not readable')
        
        return self.fget(instance)
    
    def __set__(self, instance, value):
        print('__set__ called')
        if self.fset is None:
            raise AttributeError(f'{self.prop_name} is not writable')
        
        self.fset(instance, value)

In [28]:
class Person:
    def get_name(self):
        print('get_name called')
        return getattr(self, '_name', None)
    
    def set_name(self, value):
        print('set_name called')
        self._name = value 

    name = MakeProperty(fget=get_name, fset=set_name)

In [29]:
p = Person()
p.name = 'Guido'

__set__ called
set_name called


In [30]:
p.name

__get__ called
get_name called


'Guido'

In [31]:
p.__dict__

{'_name': 'Guido'}

In [32]:
p.__dict__['name'] = 'Alex'

In [33]:
p.name

__get__ called
get_name called


'Guido'

In [34]:
p.__dict__

{'_name': 'Guido', 'name': 'Alex'}

Using the custom class as a decorator

In [40]:
class Person:

    @MakeProperty
    def age(self):
        return 500

In [41]:
p = Person()
p.age

__get__ called


500

In [42]:
class Person:

    @MakeProperty
    def age(self):
        return getattr(self, '_age', None)
    
    @age.setter
    def age(self, value):
        self._age = value


AttributeError: 'MakeProperty' object has no attribute 'setter'